# EEMD + LGBM

---

In [ ]:
# DataFrame
import pandas as pd
import numpy as np
import random
from datetime import date
import time

# Visualization
import matplotlib.pyplot as plt
import matplotlib
import warnings
warnings.simplefilter(action='ignore')

# Save the log
import os
import pickle

# EEMD
from PyEMD import EEMD

# LGBM
from lightgbm import LGBMRegressor

# Metric 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
# set the seed
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

In [ ]:
# Minus
matplotlib.rcParams['axes.unicode_minus'] = False
# font
plt.rc('font', family='Serif')

In [ ]:
def split_data(eIMF_df):
    
    train_df = eIMF_df[eIMF_df['Date']<'2022-08-01'].reset_index(drop=True)
    test_df = eIMF_df[eIMF_df['Date']>='2022-08-01'].reset_index(drop=True)
    return train_df, test_df

In [ ]:
def eemd_fit(df):
    # Define signal
    t = np.array(df['Date']) # x-axis
    s = np.array(df['y']) # y-axis

    eemd = EEMD()
    eemd.noise_seed(1234)

    eIMFs = eemd.eemd(s, t) 
    nIMFs = eIMFs.shape[0] 
    
    imfs, residue = eemd.get_imfs_and_residue()

    all_eIMFs_df = pd.DataFrame(eIMFs).transpose() 
    all_eIMFs_df[nIMFs] = residue 
    
    for column in df.drop(columns=['y']).columns:
        all_eIMFs_df[column] = df[column]
    
    # IMF & Residue 
    plt.figure(figsize=(12, nIMFs*2)) 
    for i in range(nIMFs):
        plt.subplot(nIMFs+1, 1, i+1) 
        plt.plot(df['Date'], all_eIMFs_df[i], 'g')
        plt.title('IMF '+str(i+1), fontsize=10)

    # Residue plot
    plt.subplot(nIMFs+1, 1, nIMFs+1)
    plt.plot(df['Date'], all_eIMFs_df[nIMFs], 'r')
    plt.title('Residue', fontsize=10)

    plt.tight_layout()
    plt.show()
    
    return all_eIMFs_df, nIMFs # eIMF+Residue

In [ ]:
def extract_eIMFs(all_eIMFs_df, nIMFs):
    all_eIMFs_dict = {}
    features = list(all_eIMFs_df.drop(columns = range(nIMFs+1)).columns)
 
    for i in range(nIMFs+1):
        tmp_df = all_eIMFs_df[features+[i]] 
        tmp_df.columns=features+['y'] 
        all_eIMFs_dict[f'eIMFs_{i}'] = tmp_df 
                
    return all_eIMFs_dict 

In [ ]:
def EEMD_LGBM(all_eIMFs_dict):
    
    pred_dict = {}
    
    for i in all_eIMFs_dict.keys():
        print(f'--------Total: 0~{len(all_eIMFs_dict)-1} eIMFs, Now: {i} --------')
      
        eIMF_df = all_eIMFs_dict[i]
        train_df, test_df = split_data(eIMF_df)
        predictions = []
       
        for test_date in test_df['Date']:
            # Use data up to the day before the test date to train the model
            train_until_test_df = eIMF_df[eIMF_df['Date'] < test_date]

            X = train_until_test_df.drop(columns = ['Date', 'y', 'Product'])
            y = train_until_test_df['y']
            
            lgbm_model = LGBMRegressor()
            
            # fitting the model
            best_model_fit = lgbm_model.fit(X, y)

            test_X = eIMF_df[eIMF_df['Date'] == test_date]
            test_X = test_X.drop(columns = ['Date', 'y', 'Product'])
        
            # Predict the next day's value and add it to the predictions list
            prediction = best_model_fit.predict(test_X)[0]
            predictions.append(prediction)  # replace negative predictions with 0

        # Make the Result DataFrame
        res_df = test_df.copy()
        res_df['Pred'] = predictions
        res_df.set_index('Date', inplace=True)
        # res_df: ['y'','Pred'] index='Date'
        pred_dict[i] = res_df
        
    return pred_dict

In [ ]:
def save_model(product_code, model_dict):
    today = date.today()
    folder_path = 'Saved_Model/EEMD+LGBM'
    file_name = f'{product_code}_{today.month:02d}{today.day:02d}.pkl'
    save_path = os.path.join(folder_path, file_name)
    
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    with open(save_path, 'wb') as f:
        pickle.dump(model_dict, f)
    return model_dict

In [ ]:
def load_model(file_name):
    file_path = f'Saved_Model/EEMD+LGBM/{file_name}'
    
    with open(file_path, 'rb') as file:
        model_dict= pickle.load(file)
    
    return model_dict

In [ ]:
def make_all_result_df(pred_dict):
    all_df = pd.DataFrame()
    for tmp_df in pred_dict.values():
        all_df = pd.concat([all_df, tmp_df], axis=1)
    pred_df = all_df['Pred'].sum(axis=1)
    actual_df = all_df['y'].sum(axis=1)

    all_result_df = pd.DataFrame({'Pred': pred_df, 'y': actual_df})
    all_result_df.loc[all_result_df['Pred']<0, 'Pred']=0 

    # all_res_df: ['y'','Pred'] index='Date'
    return all_result_df

In [ ]:
"""
Plot the actual vs predition and save the figure in the given directory
"""
def actual_pred_plot(product_code, pred_dict, all_result_df, metric_df):
 
    pred_dict['all_result'] = all_result_df
    save_path = os.path.join("Result", "EEMD+LGBM", product_code)

    for i, pred_df in enumerate(pred_dict.values()):
        img_n = len(pred_dict)
        title = f"Pred Actual Plot - ({i+1}/{img_n-1})'s eIMF"
        actual = pred_df['y']
        pred = pred_df['Pred']
        save_name = f'{product_code}_eIMF_{i+1}'
        
        if i == img_n-1: 
            title = f"{product_code}-All Result"
            save_name = f'{product_code}_all_result'

        plt.figure(figsize=(16, 8))
        plt.title(title, fontsize=20)
        plt.xlabel("Date", fontsize=14)
        plt.ylabel("Order Demand", fontsize=14)
        plt.plot(actual, label ='Actual', color='r')
        plt.plot(pred, label='Prediction', color='b')
        plt.legend(loc="upper right")
        
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        # save the figure
        plt.savefig(os.path.join(save_path, save_name+'.png'))
        plt.show()
        
    metric_df.to_csv(os.path.join(save_path, f'{product_code}_metric.csv'), encoding="utf-8-sig")
    all_result_df.to_csv(os.path.join(save_path, f'{product_code}_total_result.csv'), encoding="utf-8-sig")
    
    del pred_dict['all_result']
    plt.close('all') # close all figures to free up memory

In [ ]:
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / (actual+1)))

def nrmse(y_true, y_pred):
    mse = root_mean_squared_error(y_true, y_pred)
    target_mean = np.mean(y_true)
    nrmse = mse / target_mean
    return nrmse

def nmae(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    target_mean = np.mean(y_true)
    nmae = mae / target_mean
    return nmae

In [ ]:
def calculate_metrics(pred_df):
   
    metric_df = pd.DataFrame(columns=['MAPE', 'RMSE', 'MAE', 'NRMSE', 'NMAE', 'R2'])
    
    actual = pred_df['y']
    pred = pred_df['Pred']

    MAPE = mape(actual, pred) 
    RMSE = root_mean_squared_error(actual, pred)
    MAE = mean_absolute_error(actual,pred) 
    NRMSE = nrmse(actual,pred) 
    NMAE = nmae(actual,pred) 
    R2 = r2_score(actual, pred)

    tmp_df = pd.DataFrame({'MAPE':[round(MAPE, 4)],
                           'RMSE':[round(RMSE, 4)],
                           'MAE':[round(MAE, 4)],
                           'NRMSE':[round(NRMSE, 4)],
                           'NMAE':[round(NMAE, 4)],
                           'R2': [round(R2, 4)]})

    metric_df = pd.concat([metric_df, tmp_df])
    return metric_df

In [ ]:
def make_metric_df(pred_dict, all_result_df):
    metric_df = pd.DataFrame(columns=['MAPE', 'RMSE', 'MAE', 'NRMSE', 'NMAE', 'R2'])

    for i, pred_df in pred_dict.items():
        imf_df = calculate_metrics(pred_df)
        metric_df = pd.concat([metric_df, imf_df])
    
    imf_idx = pd.Index(['eIMF_'+str(i+1) for i in range(len(pred_dict))]) # changed result_dict to pred_dict
    metric_df.index = imf_idx # Assign the created index to metric_df
    metric_df = pd.concat([metric_df, calculate_metrics(all_result_df)], axis=0)
    metric_df = metric_df.rename(index={metric_df.index[-1]: 'All'})
    
    return metric_df

In [ ]:
def execute_EEMD_LGBM(df, product_code, str_code):
    start_time = time.time()

    product_df = df[df['Product']== product_code].reset_index(drop=True)

    all_eIMFs_df, nIMFs = eemd_fit(product_df)
    all_eIMFs_dict = extract_eIMFs(all_eIMFs_df, nIMFs)

    pred_dict = EEMD_LGBM(all_eIMFs_dict) #dictionary, time_steps, epochs
    all_result_df = make_all_result_df(pred_dict)

    metric_df = make_metric_df(pred_dict, all_result_df)

    actual_pred_plot(str_code, pred_dict, all_result_df, metric_df)
    
    elapsed_time_seconds = time.time() - start_time
    elapsed_time_minutes = elapsed_time_seconds / 60
    print("실행 시간: {:.2f} 분".format(elapsed_time_minutes))
    
    return metric_df, all_result_df

---

In [ ]:
df = pd.read_csv("../Data/dataset.csv")
df['Date'] = pd.to_datetime(df['Date'])
df = df.drop(columns=['년월'])

In [ ]:
set_seed(1234)
metric_df = pd.DataFrame()
result_df = pd.DataFrame()

target_code = ['Office Product', 'Packaging material', 'Pharmaceuticals']
for code in target_code:
    
    print("==================================")
    print(f"========== { str(code) } ==========")
    print("==================================")
    all_metric, all_result = execute_EEMD_LGBM(df, code, str(code))
    
    metric_df = pd.concat([metric_df, all_metric])
    result_df = pd.concat([result_df, all_result])

prod_metric_df = metric_df.loc['All']
prod_metric_df.index = target_code